# XPRESStools Notebook 
- This test notebook can be used to perform analysis on a dataset.   
- Follow the instructions above each code block to know when and how to modify aspects of the analysis    
- A block of code can be executed in Jupyter notebook by selecting the block and pressing Shift + Enter
- IMPORTANT: If you re-run a previous block of code, already created variables are stored, so it important to run this linearly    
- Please refer to the [documentation](https://xpresstools.readthedocs.io/en/latest/) for further instructions

# Preparing Data

### Import libraries 
- XPRESStools can be installed by executing ```pip install xpresstools``` in the command line   
- The ```%matplotlib inline``` command allows figures to be displayed inline in Jupyter notebook

In [ ]:
import os
import pandas as pd 
import numpy as np
import xpressplot as xp
%matplotlib inline

### Variables
- Modify variables such as data path, file names, etc here.  
- More information on how custom datasets should be formatted can be found [here](https://xpresstools.readthedocs.io/en/latest/content/general-usage.html)

In [ ]:
#Modify this path to specify file location and name 
#IMPORTANT: this path must end with a forward slash character as shown below
file_name = './tests/test_data.txt'

### Import dataset

In [ ]:
data = xp.get_df(file_name, delimiter='\t')
metadata = xp.get_info('./tests/test_info.txt')

- We can look at the data by applying the `.head()` function to a dataframe

In [ ]:
data.head()

In [ ]:
metadata.head()

### Clean up the data
- For this dataset, we only want to compare normal colon and adenomous colon expression, so we create a dataframe with just those samples and run a basic clean-up function to remove Null values, etc.   
- Print functions have also been included to show the size of the dataframe before and after this code block is run

In [ ]:
#Get normal and adenoma samples
data = xp.keep_labels(data, metadata, label_list=['ribo_untr','ribo_tm',])

#Print some info
print('Dataframe size before cleaning: ' + str(data.shape))

#Clean up dataset of missing values
data = xp.clean_df(data)

print('Dataframe size after cleaning: ' + str(data.shape))

In [ ]:
data.columns

### Rename genes
- Let's rename the gene ids to the common gene names 
- You will need an Ensembl GTF file which can be downloaded <a href='ftp://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz'>here</a> for human 
- If you are using a different model organism, you will need to download the appropriate GTF file

In [ ]:
#Location of GTF reference file
gtf_ref = '/Users/jordan/Desktop/reference/Homo_sapiens.GRCh38.96.gtf'

In [ ]:
data = xp.convert_names(
    data,
    gtf_ref)

### Sample Normalization (RNA-seq)
- If running MicroArray data, signal must be normalized using a software tool such as Alt Analyze
- If running an RNAseq dataset, samples need to be normalized based on the number of reads in each
    - See the [documentation](https://xpresstools.readthedocs.io/en/latest/content/normalize.html) for more information
    - The code blocks below will perform RPKM normalization on single-end read RNAseq data
    - The first of these code blocks will set the variable for the location and file name of the appropriate [Ensembl GTF reference file](https://uswest.ensembl.org/index.html)
- GTF file format will vary slightly from organism to organism, and at times the character sequence needed to be searched will vary. More information can be found in the [documentation](https://xpresstools.readthedocs.io/en/latest/content/normalize.html)
- If samples were sequenced in multiple batches, it is recommended using XPRESStools's [batch_normalize](https://xpresstools.readthedocs.io/en/latest/content/normalize.html#batch-normalize) function

In [ ]:
# Choose from rpm(), tpm(), rpkm(), or fpkm(). All require a GTF file aside from rpm()
data_tpm = xp.tpm(
    data, 
    gtf_ref)

In [ ]:
data_tpm.head()

In [ ]:
data_rpm = xp.rpm(
    data)

### Check the data distribution 
- As RNA-Seq data follows a negative binomial distribution, it is often helpful to perform a log<sub>10</sub> transformation of the data
- As we can see by running this block, after `rpm` normalization, the data are distributed proportionally and comparable

In [ ]:
log_data = np.log10(data_rpm + 1)

xp.check_samples(log_data)

### Gene Scaling
- For many analyses, it is important that genes be scaled similarly in order to make their comparison compatible
- XPRESStools will perform standard scaling on gene data. This will result in each gene having a mean=0 and a standard deviation=1

In [ ]:
data_scaled, data_labeled = xp.prep_data(data_rpm, metadata) 

### Prepare plotting features
- XPRESStools often gives one the option to color by sample type. Doing so requires a color palette dictionary, like the one below
- Colors can be specified using common names or using another Matplotlib-compatible format

In [ ]:
colors = {'ribo_tm': (0.8705882352941177, 0.5607843137254902, 0.0196078431372549),
        'ribo_untr': (0.00784313725490196, 0.6196078431372549, 0.45098039215686275)}

# Analyzing Data

### Single-gene analysis
- Use: Explore how expression of a particular gene varies across sample types

<u><b>Plot data</b></u>
- In the example below, we use the previous data and metadata, along with the corresponding color palette
- In this example we will look at how expression of <b>ATF4</b>, a stress-responsive transcription factor
- In this example, we have also added the ```order``` argument to ensure samples are ordered as we wish

In [ ]:
xp.gene_overview(data_rpm, metadata, 'ATF4', colors, order=['ribo_untr','ribo_tm'])

### Multi-gene analysis
- Use: Explore expression of a sub-set of genes of interest

<u><b>Generate gene lists</b></u>
- Gene lists can be imported or specified as a variable for later use
- This list can be modified to fit the user's specific needs

In [ ]:
genes = ['ATF4',"ATF5",'DDIT3','PPP1R15A']

<u><b>Plot data</b></u>
- Next, we'll perform multi-gene analysis on a subset of genes we think might be interesting in the context of colon cancer
- In this example we will explore the expression of the genes required to form the HoxD
- It is recommended that <b>genes are uniformly scaled</b> in order to make their expression levels cross-compatible
- Lactate dehydrogenase gene products convert pyruvate to lactate and appear to show sligh up-regulation. As the mitochondrial pyruvate carrier appears to be downregulated, this makes sense, as the complex will decrease import of pyruvate into the mitochondria and lead to accumulations in the cytosol

In [ ]:
xp.multigene_overview(data_scaled, metadata, 
                      palette=colors,
                      gene_list=genes)

### Heatmaps
- Use: Compare a gene cohort or all genes across samples to identify expression patterns

<u><b>Prepare gene labels metadata</b></u>
- In this example, we will supply the sample metadata, as well as gene metadata to label the gene axis

In [ ]:
#Set gene group color palette
gene_colors = {'ATF4': (0.5725490196078431, 0.5843137254901961, 0.5686274509803921),
                'ATF5': (0.8705882352941177, 0.5607843137254902, 0.0196078431372549)}

#Create gene group metadata
gene_groups = np.array([['ATF4','ISR'],
                  ['ATF5','ISR'],
                  ['DDIT3','ISR'],
                  ['SLC1A1','Other']])
gene_groups = pd.DataFrame({0:gene_groups[:,0],1:gene_groups[:,1]})

<u><b>Plot data</b></u>
- From this analysis, it appears that the LDH genes and the MPC genes cluster together and that these two sets of genes alone are able to cluster normal versus adenomous tissue fairly well

In [ ]:
xp.heatmap(data_scaled, metadata, 
           sample_palette=colors, 
           gene_palette=gene_colors, gene_info=gene_groups, 
           gene_list=['ATF4','ATF5','DDIT3','SLC1A1'], 
           row_cluster=True, col_cluster=True,
           figsize=(10,3))

# Scatter Plots and Linear Regression
- Use: Plot two arrays data, such as the expression of two genes from the dataset and determine if there is a linear relationship between the two arrays 

<u><b>Plot data</b></u>
- From the previous example, it appears there is a fairly anti-correlary relationship between MPC and LDH. We would now like to quantify this relationship between two of the genes
- The linear regression states that the relationship has an r-value=```-0.48```, confirming our hypothesis of a anti-correlary relationship. The value of ```0.48``` indicates a moderate level of correlation

In [ ]:
xp.scatter(data, metadata, 'SLC1A1', 'ATF4', 
           palette=colors, 
           add_linreg=True,
           alpha=0.9,
           figsize=(4,4))

### Volcano Plots
- As RNA-Seq data is not normally distributed, we need to input properly processed data 
- We will use DESeq2 prepared data, which is accessible through `XPRESSpipe diffex`

<u><b>Plot data</b></u>
- From this analysis, while the changes in these genes are significant, they are perhaps not very large changes in expression
- Labeling the highlighted points is a two step process, as outlined in the code blocks below

In [ ]:
#We'll use this step to get the gene coordinates
xp.rna_volcano(
    './tests/test_deseq2_output.txt',
    highlight_points=[['ATF4','ATF5','DDIT3'],['SLC1A1','POMGNT1']],
    highlight_color=['green','purple'],
    y_threshold=1,
    x_threshold=[-1,1])                                      
           

# Principle Component Analysis
- Use: Deconvolute high-dimensional data

<u><b>Prepare gene set</b></u>
- Let's see how sufficient solute transporter expression levels are to delineate different sample types

In [ ]:
slc = []
slc = [x for x in data_scaled.index.tolist() if 'SLC' in x]

<u><b>Plot data</b></u>
- Let's plot the gene subset

In [ ]:
xp.pca(data_scaled, metadata, colors, 
       gene_list=slc,
       grouping='samples', #this indicates we want to compare samples
       n_components=2, # Since we have a small number of samples, we need to modify this to n_samples > n_components
       figsize=(4,4))

- We can also consider all gene expression to see how it delineates sample type
- In this case, it looks like gene expression alone is enough to cluster these samples (good thing, since they one type is cancer and the other is normal!)

In [ ]:
xp.pca(data_scaled, metadata, colors, 
       figsize=(4,4),
      n_components=2)